In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.applications import NASNetLarge

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/seg_train/seg_train'
test_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/seg_test/seg_test'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 10,
                                   shear_range = 0.3,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   zoom_range = 0.25)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
train_data = train_datagen.flow_from_directory(train_path,
                                               batch_size = 64,
                                               target_size = (224, 224),
                                               shuffle = True)
test_data = test_datagen.flow_from_directory(test_path,
                                             batch_size = 64,
                                             target_size = (224, 224),
                                             shuffle = False)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [5]:
baseModel = NASNetLarge(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
baseModel.trainable = True

model = Sequential()
model.add(baseModel)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(216, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(6, activation = 'softmax'))

In [6]:
adam = Adam(learning_rate = 0.0001)

model.compile(optimizer = adam, loss= 'categorical_crossentropy', metrics = ['accuracy'])

In [7]:
history = model.fit(train_data, epochs = 5, batch_size = 64, validation_data = test_data)

Epoch 1/5


Exception ignored in: <function _xla_gc_callback at 0x7c415f271cf0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Doğruluğu')
plt.ylabel('Doğruluk')
plt.xlabel('Epoch')
plt.legend(['Eğitim', 'Test'], loc = 'upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Kaybı')
plt.ylabel('Kayıp')
plt.xlabel('Epoch')
plt.legend(['Eğitim', 'Test'], loss = 'upper left')
plt.show()

In [ ]:
import random
from keras.preprocessing import image
import os

test_image_paths = [os.path.join(test_path, category, random.choice(os.listdir(os.path.join(test_path, category))))
                    for category in os.listdir(test_path)]

random_image_path = random.choice(test_image_paths)

In [ ]:
img = image.load_img(random_image_path, target_size = (224, 224))
img_array = image.img_to_array(img)
img_aray = np.exand_dims(img_array, axis = 0)
img_array /= 255.

In [ ]:
prediction = model.predict(img_array)
predicted_classes = np.argmax(prediction)

In [ ]:
class_labels = list(train_data.class_indices.kes())

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
print("Tahmin edilen sınıf:", class_labels[predicted_classes])
print("Tüm sınıf olasılıkları:", prediction)

In [ ]:
loss, accuracy = model.evaluate(test_data)

print('Modelin doğruluk oranı: ', accuracy)